In [ ]:
from llm_classifier import LLM_classifier

In [ ]:
classifier = LLM_classifier("mistralai/Mistral-7B-Instruct-v0.2", "ronunes/anacreontea")

In [ ]:
def clean(text, query):
    punc = '''!()[]{};:'"\,<>./?@#$%^&*_~'''
    
    text = text.replace("\n", "")
    text = text.replace(query, "")
                        
    for ele in text:
        if ele in punc:
            text = text.replace(ele, "")
            
    return text

def find_conclusion(text, query):
    lines = text.split('\n')
    conclusion_text = ""

    for line in lines:
        if line.startswith(query):
            conclusion_text= line
            break

    return conclusion_text

In [ ]:
def generate_prompt(sentence: str, use_definitions: bool, type: str) -> str:
    persona = """Persona:
Você é um pesquisador especialista em literatura grega antiga com foco nos textos do corpus das anacreônticas.

Instrução:
Faça sua escolha baseado no que melhor representa a temática do texto. Utilize o nome da mesma forma que está no contexto.
"""

    labels = classifier.labels_topos
    definitions = classifier.definitions_topos
    
    context = ""
    
    if use_definitions:           
        context = f"""
Nome de cada {type} e seu contexto:
{classifier.get_definitions(labels, definitions)}
"""
    else:
        context = f"""
Lista contextual de cada {type} possível: {classifier.list_to_string_with_quotes(labels)}
"""

    anacreontic = f'''
Anacreôntica:
"""
{sentence}
"""
'''

    question = f"""
Pergunta: Por favor, me informe qual {type} está presente nesta anacreôntica e qual é o mais representativo. 
Resposta: Vamos pensar passo a passo."""

    prompt = ("\n").join([persona, context, anacreontic, question])
    
    return prompt

In [ ]:
def generate(input_text, max_new_tokens):
    inputs = classifier.LLM_tokenizer(input_text, return_tensors="pt")
    inputs = inputs.to('cuda')
    
    outputs = classifier.LLM_model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=classifier.LLM_model.config.eos_token_id, output_scores=True, return_dict_in_generate=True)
    resp = classifier.LLM_tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

    return resp

In [ ]:
topos_list = []
topos_prompt_list = []

for idx, sentence in enumerate(classifier.dataset["train"]["portuguese_text"]):
    input_text = generate_prompt(sentence, False, "topos")
    resp = generate(input_text, 800)
    
    resp_final = f"""{resp}
Nome do tópos representativo:"""
    
    resp_final = generate(resp_final, 40)
    
    result = find_conclusion(resp_final, "Nome do tópos representativo: ")
    result = clean(result, "Nome do tópos representativo: ")
    
    topos_list.append(result)
    topos_prompt_list.append(resp_final)
    
    print(idx, result)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(classifier.dataset["train"]["topos"], topos_list))